In [4]:
import sys

path_to_model_directory = '../model'

# Add this path to sys.path
if path_to_model_directory not in sys.path:
    sys.path.append(path_to_model_directory)

# Now you can import your class
from PreTrainer import PreTrainer

In [5]:
# Load the PreTrainer
BATCH_SIZE = 32
LR = 0.001
EPOCHS = 10
train_path = '/home/florsanders/adl_ai_tennis_coach/data/tenniset/shot_labels/train'
val_path = '/home/florsanders/adl_ai_tennis_coach/data/tenniset/shot_labels/val'
model_config_file = '/home/tawab/e6691-2024spring-project-TECO-as7092-gyt2107-fps2116/src/model/configs/default.yaml'

In [7]:
# Trainer Setup
trainer = PreTrainer(
    batch_size=BATCH_SIZE,
    lr=LR,
    epochs=EPOCHS,
    train_path=train_path,
    val_path=val_path,
    model_config_path=model_config_file
)

Skipping V006_0068: Incorrect dimensions - positions_2d (0, 2), poses_3d (0,)
Skipping V006_0179: Incorrect dimensions - positions_2d (0, 2), poses_3d (0,)
Skipping V007_0183: Incorrect dimensions - positions_2d (0, 2), poses_3d (0,)
Skipping V007_0184: Incorrect dimensions - positions_2d (0, 2), poses_3d (0,)
Skipping V008_0003: Data file not found.
Skipping V008_0056: Incorrect dimensions - positions_2d (0, 2), poses_3d (0,)
Skipping V008_0156: Incorrect dimensions - positions_2d (0, 2), poses_3d (0,)
Skipping V009_0017: Incorrect dimensions - positions_2d (0, 2), poses_3d (0,)
Skipping V009_0924: Incorrect dimensions - positions_2d (0, 2), poses_3d (0,)
Skipping V009_0947: Incorrect dimensions - positions_2d (0, 2), poses_3d (0,)
Skipping V009_0948: Incorrect dimensions - positions_2d (0, 2), poses_3d (0,)
Skipping V009_1045: Incorrect dimensions - positions_2d (0, 2), poses_3d (0,)
Skipping V009_1281: Incorrect dimensions - positions_2d (0, 2), poses_3d (0,)
Skipping V009_1282: Inc

In [8]:
trainer.train()

Epoch 1, Loss: 4.3892199590172565
Test Loss: 3.8792288303375244
Epoch 2, Loss: 3.8762943543178934
Test Loss: 3.841962734858195
Epoch 3, Loss: 3.7522318296029535
Test Loss: 3.5355200502607556
Epoch 4, Loss: 2.0885988826483066
Test Loss: 0.8653882013426887
Epoch 5, Loss: 0.7307362598432622
Test Loss: 0.7242493993706174
Epoch 6, Loss: 0.67309780691711
Test Loss: 0.6832493543624878
Epoch 7, Loss: 0.6437299259951417
Test Loss: 0.6656446042988036
Epoch 8, Loss: 0.6231982947235376
Test Loss: 0.6091782632801268
Epoch 9, Loss: 0.5776030882983141
Test Loss: 0.5855830427673128
Epoch 10, Loss: 0.555413370401087
Test Loss: 0.5524877508481344
